In [ ]:
!pip install pylorentz --user

In [1]:
import uproot 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from pylorentz import Momentum4
from pylorentz import Position4

In [ ]:
# tree_tt = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_AllHiggs_tt.root")["ntuple"]
tree_tt = uproot.open("/eos/user/s/stcheung/SWAN_projects/Masters_CP/MVAFILE_AllHiggs_tt.root")["ntuple"]
print('loaded root file')
variables = [
            "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand",
            "aco_angle_1", "aco_angle_5", "aco_angle_6", "aco_angle_7", 
            "mva_dm_1","mva_dm_2",
            "tau_decay_mode_1","tau_decay_mode_2",
            "ip_x_1", "ip_y_1", "ip_z_1", "ip_x_2", "ip_y_2", "ip_z_2", # ignore impact parameter for now
            "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", 
            "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", 
            "pi0_E_1", "pi0_px_1", "pi0_py_1", "pi0_pz_1",
            "pi0_E_2", "pi0_px_2", "pi0_py_2", "pi0_pz_2", 
            "y_1_1", "y_1_2",
            'met', 'metx', 'mety',
            'metCov00', 'metCov01', 'metCov10', 'metCov11'
            'sv_x_1', 'sv_y_1', 'sv_z_1', 'sv_x_2', 'sv_y_2','sv_z_2'
        ]

variables += ["gen_nu_p_1", "gen_nu_phi_1", "gen_nu_eta_1", #leading neutrino, gen level
            "gen_nu_p_2", "gen_nu_phi_2", "gen_nu_eta_2" #subleading neutrino, gen level
             ]

df_tt = tree_tt.pandas.df(variables)
print('loaded df')
# select only rho-rho events
df_rho = df[(df['mva_dm_1']==1) & (df['mva_dm_2']==1) & (df["tau_decay_mode_1"] == 1) & (df["tau_decay_mode_2"] == 1)]
# drop unnecessary labels 
df = df_rho.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
print('finished rho-rho loading')

In [ ]:
# gen level data
tree_tt_gen = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_GEN_AllHiggs_tt.root")["ntuple"]
variables_gen = [
            "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand",
            "dm_1", "dm_2",
            "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", # charged pion 1
            "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", # charged pion 2
            "pi0_E_1", "pi0_px_1", "pi0_py_1", "pi0_pz_1", # neutral pion 1
            "pi0_E_2", "pi0_px_2", "pi0_py_2", "pi0_pz_2", # neutral pion 2,
            'metx', 'mety',
            ]
df_tt_gen = tree_tt_gen.pandas.df(variables_gen)
df_rho = df_tt_gen[(df_tt_gen['dm_1']==1) & (df_tt_gen['dm_2']==1)]

In [ ]:
df_tt_gen.to_pickle('./df_tt_gen.pkl')

In [ ]:
df_tt_gen = pd.read_pickle('./df_tt_gen.pkl')

In [ ]:
df_tt.to_pickle('./df_tt.pkl')

In [2]:
df_tt = pd.read_pickle('df_tt.pkl')

In [3]:
channel = 'rho_rho'

In [4]:
# reco level data
if channel == 'rho_rho':
    df_rho = df_tt[(df_tt['mva_dm_1']==1) & (df_tt['mva_dm_2']==1) & (df_tt["tau_decay_mode_1"] == 1) & (df_tt["tau_decay_mode_2"] == 1)]
    df = df_rho.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
elif channel == 'rho_a1':
    df_rho_a1 = df_tt[(df_tt['mva_dm_1']==1) & (df_tt['mva_dm_2']==10) & (df_tt["tau_decay_mode_1"] == 1)]
    df = df_rho_a1.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
elif channel == 'a1_a1':
    df_a1_a1 = df_tt[(df_tt['mva_dm_1']==10) & (df_tt['mva_dm_2']==10)]
    df = df_a1_a1.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)    
else:
    print('CHANNEL not understood!')

In [ ]:
# gen level data NEED TO CHECK FOR TAU DECAY MODE LABEL
if channel == 'rho_rho':
    df_rho = df_tt_gen[(df_tt_gen['dm_1']==1) & (df_tt_gen['dm_2']==1)]
    df = df_rho.drop(["dm_1","dm_2","wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
elif channel == 'rho_a1':
    df_rho_a1 = df_tt_gen[(df_tt_gen['dm_1']==1) & (df_tt_gen['dm_2']==10)]
    df = df_rho_a1.drop(["dm_1","dm_2","wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
elif channel == 'a1_a1':
    df_a1_a1 = df_tt_gen[(df_tt_gen['dm_1']==10) & (df_tt_gen['dm_2']==10)]
    df = df_a1_a1.drop(["dm_1","dm_2","wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
else:
    print('CHANNEL not understood!')
df['met'] = np.sqrt(df['metx']**2+df['mety']**2)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# y = (~(df_rho["rand"]<df_rho["wt_cp_ps"]/2).to_numpy()).astype(int)
# np.save('./potential_2016/y_kristof', y, allow_pickle=True)

In [5]:
N = len(df['metx'])
met_x = Momentum4(df['metx'], np.zeros(N), np.zeros(N), np.zeros(N))
met_y = Momentum4(df['mety'], np.zeros(N), np.zeros(N), np.zeros(N))
met = Momentum4(df['met'], np.zeros(N), np.zeros(N), np.zeros(N))

In [6]:
pi_1 = Momentum4(df['pi_E_1'], df["pi_px_1"], df["pi_py_1"], df["pi_pz_1"])
pi_2 = Momentum4(df['pi_E_2'], df["pi_px_2"], df["pi_py_2"], df["pi_pz_2"])
pi0_1 = Momentum4(df['pi0_E_1'], df["pi0_px_1"], df["pi0_py_1"], df["pi0_pz_1"])
pi0_2 = Momentum4(df['pi0_E_2'], df["pi0_px_2"], df["pi0_py_2"], df["pi0_pz_2"])
rho_1 = pi_1 + pi0_1
rho_2 = pi_2 + pi0_2
# rho_1.m is invariant mass of rhos
# boost into rest frame of resonances
rest_frame = pi_1 + pi_2 + pi0_1 + pi0_2
boost = Momentum4(rest_frame[0], -rest_frame[1], -rest_frame[2], -rest_frame[3])
pi_1_boosted = pi_1.boost_particle(boost)
pi_2_boosted = pi_2.boost_particle(boost)
pi0_1_boosted = pi0_1.boost_particle(boost)
pi0_2_boosted = pi0_2.boost_particle(boost)
rho_1_boosted = pi_1_boosted + pi0_1_boosted
rho_2_boosted = pi_2_boosted + pi0_2_boosted
# boost MET
met_x_boosted = met_x.boost_particle(boost)
met_y_boosted = met_y.boost_particle(boost)
met_boosted = met.boost_particle(boost)

In [7]:
# rotations
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix
    
pi_1_boosted_rot = []
pi_2_boosted_rot = []
pi0_1_boosted_rot = []
pi0_2_boosted_rot = []
rho_1_boosted_rot = []
rho_2_boosted_rot = []
for i in range(pi_1_boosted[:].shape[1]):
    rot_mat = rotation_matrix_from_vectors(rho_1_boosted[1:, i], [0,0,1])
    pi_1_boosted_rot.append(rot_mat.dot(pi_1_boosted[1:, i]))
    pi0_1_boosted_rot.append(rot_mat.dot(pi0_1_boosted[1:, i]))
    pi_2_boosted_rot.append(rot_mat.dot(pi_2_boosted[1:, i]))
    pi0_2_boosted_rot.append(rot_mat.dot(pi0_2_boosted[1:, i]))
    rho_1_boosted_rot.append(rot_mat.dot(rho_1_boosted[1:, i]))
    rho_2_boosted_rot.append(rot_mat.dot(rho_2_boosted[1:, i]))
    if i%100000==0:
        print('finished getting rotated 4-vector', i)

finished getting rotated 4-vector 0
finished getting rotated 4-vector 100000
finished getting rotated 4-vector 200000
finished getting rotated 4-vector 300000
finished getting rotated 4-vector 400000
finished getting rotated 4-vector 500000
finished getting rotated 4-vector 600000
finished getting rotated 4-vector 700000
finished getting rotated 4-vector 800000
finished getting rotated 4-vector 900000


In [8]:
np.save('reco_info/rho_rho/pi_1_boosted_rot.npy', pi_1_boosted_rot, allow_pickle=True)
np.save('reco_info/rho_rho/pi_2_boosted_rot.npy', pi_2_boosted_rot, allow_pickle=True)
np.save('reco_info/rho_rho/pi0_1_boosted_rot.npy', pi0_1_boosted_rot, allow_pickle=True)
np.save('reco_info/rho_rho/pi0_2_boosted_rot.npy', pi0_2_boosted_rot, allow_pickle=True)

In [9]:
E_miss_boosted = met_boosted[0]
E_miss_x_boosted = met_x_boosted[0]
E_miss_y_boosted = met_y_boosted[0]

In [ ]:
# to_load = []
# to_load_names = ["pi_1_transformed", "pi_2_transformed", "pi0_1_transformed", "pi0_2_transformed", "rho_1_transformed", "rho_2_transformed", "aco_angle_1", "y_1_1", "y_1_2", "m_1", "m_2", "w_a", "w_b"]
# for i in range(len(to_load_names)):
#     to_load.append(np.load(f'potential_2016/{to_load_names[i]}.npy', allow_pickle=True))
# pi_1_transformed, pi_2_transformed, pi0_1_transformed, pi0_2_transformed, rho_1_transformed, rho_2_transformed, aco_angle_1, y_1_1, y_1_2, m_1, m_2, w_a, w_b = to_load

In [ ]:
# pi_1_boosted_rot = pi_1_transformed[:, 1:]
# pi_2_boosted_rot = pi_2_transformed[:, 1:]
# pi0_1_boosted_rot = pi0_1_transformed[:, 1:]
# pi0_2_boosted_rot = pi0_2_transformed[:, 1:]
# rho_1_boosted_rot = rho_1_transformed[:, 1:]
# rho_2_boosted_rot = rho_2_transformed[:, 1:]

In [10]:
higgs = rho_1_boosted + rho_2_boosted
m_tau = 1.776
m_rho = 0.7754
m_higgs = 125.18

In [11]:
E_miss = df['met'].to_numpy()
E_miss_x = df['metx'].to_numpy()
E_miss_y = df['mety'].to_numpy()

In [12]:
rho_1_boosted_rot = np.array(rho_1_boosted_rot)
rho_2_boosted_rot = np.array(rho_2_boosted_rot)

In [13]:
def calc_alpha(mode):
    if mode == 1:
        alpha_2 = (E_miss_y*rho_1[1]-E_miss_x*rho_1[2])/(rho_2[2]*rho_1[1]-rho_2[1]*rho_1[2])
        alpha_1 = (E_miss_x - alpha_2*rho_2[1])/rho_1[1]
    elif mode == 2:
        alpha_1 = (E_miss_y*rho_2[1]-E_miss_x*rho_2[2])/(rho_1[2]*rho_2[1]-rho_1[1]*rho_2[2])
        alpha_2 = (m_higgs**2/2 - m_tau**2)/(rho_1[0]*rho_2[0]-rho_1[1]*rho_2[1]-rho_1[2]*rho_1[2]-rho_1[3]*rho_1[3])/(1+alpha_1) - 1
    elif mode == 3:
        alpha_2 = (E_miss_y*rho_1[1]-E_miss_x*rho_1[2])/(rho_2[2]*rho_1[1]-rho_2[1]*rho_1[2])
        alpha_1 = (m_higgs**2/2 - m_tau**2)/(rho_1[0]*rho_2[0]-rho_1[1]*rho_2[1]-rho_1[2]*rho_1[2]-rho_1[3]*rho_1[3])/(1+alpha_2) - 1
    else:
        raise InputError('incorrect mode in parameters')
    return alpha_1, alpha_2

In [14]:
def evaluate(var, mode=0):
    if mode == 0:
        print(f"Fraction of < 0 in {var}: {(eval(var)<0).sum()/len(eval(var)):.3f}, {(eval(var)>0).sum()} values left")
    elif mode == 1:
        print(f'Fraction of NaNs in {var}: {np.isnan(eval(var)).sum()/len(eval(var)):.3f}, {len(eval(var))-np.isnan(eval(var)).sum()} values left')

In [ ]:
np.random.multivariate_normal()

In [ ]:
# calculate alphas from gaussian distribution
def getAlpha(mode, termination=1000):
    alpha_1, alpha_2 = calc_alpha(mode)
    if alpha_1 < 0:
        pass

In [15]:
# use lab frame calculate alphas
alpha_1, alpha_2 = calc_alpha(1)

evaluate('alpha_1')
evaluate('alpha_2')
# # set negative alpha to None
# alpha_1[alpha_1<0] = None
# alpha_2[alpha_2<0] = None

# z component of neutrino mom (use boosted and rot frame now)
p_z_nu_1 = alpha_1*rho_1_boosted_rot[:, 2]
p_z_nu_2 = alpha_2*rho_2_boosted_rot[:, 2]
E_nu_1 = (m_tau**2 - rho_1_boosted[0]**2 + rho_1_boosted_rot[:, 2]**2 + 2*p_z_nu_1*rho_1_boosted_rot[:, 2])/(2*rho_1_boosted[0])
E_nu_2 = (m_tau**2 - rho_2_boosted[0]**2 + rho_2_boosted_rot[:, 2]**2 + 2*p_z_nu_2*rho_2_boosted_rot[:, 2])/(2*rho_2_boosted[0])
p_t_nu_1 = np.sqrt(E_nu_1**2 - p_z_nu_1**2)
p_t_nu_2 = np.sqrt(E_nu_2**2 - p_z_nu_2**2)

# need to check alpha and E are positive - seen total 17% of values rej (right now only see from rho-rho decay chain)

evaluate('E_nu_1')
evaluate('E_nu_2')
evaluate('p_t_nu_1', 1)
evaluate('p_t_nu_2', 1)

# # set negative alphas and E to None values
# alpha_1[alpha_1<0] = None
# alpha_2[alpha_2<0] = None

# make negative p_t values to 0
p_t_nu_1[np.isnan(p_t_nu_1)] = 0
p_t_nu_2[np.isnan(p_t_nu_2)] = 0

Fraction of < 0 in alpha_1: 0.459, 540254 values left
Fraction of < 0 in alpha_2: 0.435, 563774 values left
Fraction of < 0 in E_nu_1: 0.458, 540637 values left
Fraction of < 0 in E_nu_2: 0.435, 564082 values left
Fraction of NaNs in p_t_nu_1: 0.495, 504333 values left
Fraction of NaNs in p_t_nu_2: 0.480, 519486 values left


<ipython-input-15-e3a375a7aa8f>:15: RuntimeWarning: invalid value encountered in sqrt
  p_t_nu_1 = np.sqrt(E_nu_1**2 - p_z_nu_1**2)
<ipython-input-15-e3a375a7aa8f>:16: RuntimeWarning: invalid value encountered in sqrt
  p_t_nu_2 = np.sqrt(E_nu_2**2 - p_z_nu_2**2)


In [ ]:
idx_a_1 = np.where(alpha_1<0)[0]
idx_a_2 = np.where(alpha_2<0)[0]
idx_E_1 = np.where(E_nu_1<0)[0]
idx_E_2 = np.where(E_nu_2<0)[0]
i1 = np.union1d(idx_a_1, idx_E_1)
i2 = np.union1d(idx_a_2, idx_E_2)
print(f'Events left from tau_1: {alpha_1.shape[0]-i1.shape[0]}')
print(f'Events left from tau_2: {alpha_2.shape[0]-i2.shape[0]}')

In [ ]:
# need to remove events that are rejected (for now)
df['alpha_1'] = alpha_1
df['alpha_2'] = alpha_2
df['E_nu_1'] = E_nu_1
df['E_nu_2'] = E_nu_2
df['p_t_nu_1'] = p_t_nu_1
df['p_t_nu_2'] = p_t_nu_2
df_red = df[(df['alpha_1']>0) & (df['alpha_2']>0) & (df['E_nu_1']>0) & (df['E_nu_2']>0)].reset_index(drop=True)

df_red.head()

In [ ]:
1-df_red.shape[0]/df.shape[0]

## results

### for approx 1
- rho-rho channel: 0.459/0.453 alpha - 61.6% total rej
- rho-a1 channel: 0.461/0.502 alpha - 65.5% total rej
- a1-a1 channel: 0.507/0.489 alpha - 68.5% total rej

In [ ]:
# from paper - doesn't work because SV isn't always defined
c = 299792458
t_flight = 87e-6/c
# calculate in lab frame
p_x_tau_1 = -m_tau*df['sv_x_1']/t_flight
p_y_tau_1 = -m_tau*df['sv_y_1']/t_flight
p_y_tau_1 = -m_tau*df['sv_y_1']/t_flight
p_x_tau_2 = -m_tau*df['sv_x_2']/t_flight
p_y_tau_2 = -m_tau*df['sv_y_2']/t_flight
p_y_tau_2 = -m_tau*df['sv_y_2']/t_flight
# need to convert units of p
E_tau_1 = np.sqrt(p_x_tau_1**2+p_y_tau_1**2+p_z_tau_1**2+m_tau**2)
E_tau_2 = np.sqrt(p_x_tau_2**2+p_y_tau_2**2+p_z_tau_2**2+m_tau**2)

# rho reconstruction check

In [ ]:
plt.rcParams.update({'font.size': 14, "figure.figsize": (10,6)})
n_1, bins_1, patches_1 = plt.hist(rho_1.m, bins=2000, label=f"Peak rho_1 mass: {bins_1[np.where(n_1==n_1.max())][0]:.4f} Gev")
n_2, bins_2, patches_2 = plt.hist(rho_2.m, bins=2000, label=f"Peak rho_2 mass: {bins_2[np.where(n_2==n_2.max())][0]:.4f} Gev")

plt.xlim([0,5])
plt.legend()
plt.ylabel('Freq')
plt.xlabel('Mass (Gev)')
plt.tight_layout()
plt.savefig('rho_reconstruction.PNG')
plt.show()